# Installing *torch-summary*

In [ ]:
!pip install torch-summary

# Importing necessary libraries

In [ ]:
import scipy
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn.functional as F
from torchsummary import summary
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Checking if GPU is available

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print("GPU Runtime Detected")

else:
  device = torch.device("cpu")
  print("No GPU Found - CPU Runtime")

# Defining the *splitter* function

In [ ]:
def splitter(array, win_len, hop_len, return_df = True):
   N = array.shape[0]
   m = 0
   ids = []
   while m + win_len < N:
      ids.append([m, m + win_len])
      m = m + hop_len
      
   if return_df:
      return pd.DataFrame([array[i[0]: i[1]] for i in ids])
   else:
      return np.array([array[i[0]: i[1]] for i in ids])

# Loading the Dataset

In [ ]:
file = "../input/engine-acoustic-emissions/dataset.mat"
dataset = scipy.io.loadmat(file)
dataset.keys()

# Preprocessing

## Splitting the original Dataset

In [ ]:
win_len, hop_len = 8000, 1000
df_list = []

for state in ['normal', 'inner', 'roller', 'outer']:
    temp_df = splitter(dataset[state].reshape((-1,)), win_len, hop_len, return_df = True)
    temp_df['state'] = state
    df_list.append(temp_df)

df = pd.concat(df_list).reset_index(drop = True)
df

## Train/Test Splitting

In [ ]:
df_train, df_test = train_test_split(df, test_size = 0.3, random_state = 42)

## Feature/Target Declaration

In [ ]:
x_train, x_test = df_train.iloc[:, :-1], df_test.iloc[:, :-1]
y_train, y_test = df_train.iloc[:, -1], df_test.iloc[:, -1]

## Feature Scaling

In [ ]:
def time_series_scaler(df):

  array_mu = np.mean(df.to_numpy(), axis = 1)
  array_sig = np.std(df.to_numpy(), axis = 1)

  array_scaled = (np.subtract(df.to_numpy().transpose(), array_mu) / array_sig).transpose()

  return array_scaled

In [ ]:
x_train_scaled, x_test_scaled = time_series_scaler(x_train), time_series_scaler(x_test)

## Target Encoding

In [ ]:
labelencoder= LabelEncoder()
y_train_encoded = labelencoder.fit_transform(y_train.to_numpy())
y_test_encoded = labelencoder.transform(y_test.to_numpy())

# Training

## Model Declaration

In [ ]:
class Classifier(torch.nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        self.conv = torch.nn.Conv1d(in_channels = 1, out_channels = 5, kernel_size = 500)
        self.avgPool = torch.nn.AvgPool1d(kernel_size = 100)
        self.fc = torch.nn.Linear(5 * 75, 4)

    def forward(self, x):

        z = self.conv(x)
        z = torch.tanh(z)
        z = self.avgPool(z)

        z = z.view(-1, 5 * 75)

        z = self.fc(z)

        return z

In [ ]:
model = Classifier().to(device)
summary(model, (1, 8000));

## Fitting the Model

In [ ]:
x_train_train, x_train_validation, y_train_train, y_train_validation = train_test_split(x_train_scaled, y_train_encoded, test_size = 0.25)

x_train_train = x_train_train.reshape(-1, 1, 8000)
x_train_validation = x_train_validation.reshape(-1, 1, 8000)

x_train_VAR = torch.autograd.Variable(torch.Tensor(x_train_train).float()).to(device)
y_train_VAR = torch.autograd.Variable(torch.LongTensor(y_train_train)).to(device)
x_valid_VAR = torch.autograd.Variable(torch.Tensor(x_train_validation).float()).to(device)
y_valid_VAR = torch.autograd.Variable(torch.LongTensor(y_train_validation)).to(device)

In [ ]:
lr = 0.005
ep = 150

model = Classifier().to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr = lr,
                             weight_decay = lr / ep)

losses = []
valid_losses = []
accs = []
valid_accs = []



for epoch in range(ep):

  # validation step
  valid_loss = criterion(model(x_valid_VAR), y_valid_VAR).item()
  valid_losses.append(valid_loss)
  valid_acc = accuracy_score(y_train_validation, np.argmax(model(x_valid_VAR).cpu().detach().numpy(), axis = 1))
  valid_accs.append(valid_acc)

  # training step
  optimizer.zero_grad()
  loss = criterion(model(x_train_VAR), y_train_VAR)
  acc = accuracy_score(y_train_train, np.argmax(model(x_train_VAR).cpu().detach().numpy(), axis = 1))
  accs.append(acc)
  losses.append(loss.item())
  loss.backward()
  optimizer.step()
  print(f"Epoch {epoch+1}, loss: {np.round(loss.item(), 4)}  , Vloss: {np.round(valid_loss, 4)}, acc: {np.round(acc, 4)}, Vacc: {np.round(valid_acc, 4)}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16,4))
fig.suptitle('Deep Learning Model Training Process')
axes[0].plot(losses, label='Training Loss')
axes[0].plot(valid_losses, label='Validation Loss')
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss Vs. Epochs')
axes[0].legend()

axes[1].plot(accs, label='Training Accuracy')
axes[1].plot(valid_accs, label='Validation Accuracy')
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Accuracy Vs. Epochs')
axes[1].legend()
plt.show()

# Evaluation

## Test-set Accuracy

In [ ]:
x_test_VAR = torch.autograd.Variable(torch.Tensor(x_test_scaled.reshape(-1, 1, 8000)).float()).to(device)
testing_acc = accuracy_score(y_test_encoded, np.argmax(F.softmax(model(x_test_VAR), dim = 0).cpu().detach().numpy(), axis = 1))
print('Held-out Test-set Accuracy: ', round(testing_acc, 4))

## Test-set Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_test_pred = np.argmax(F.softmax(model(x_test_VAR), dim = 0).cpu().detach().numpy(), axis = 1)
y_test_pred_decoded = labelencoder.inverse_transform(y_test_pred)

matrix = confusion_matrix(y_test_encoded, y_test_pred)

ax = sns.heatmap(matrix, annot=True, fmt='d', cbar = True, square = True, cmap = 'Blues')
ax.set_xlabel("Predicted", fontsize=14, labelpad=20)
ax.xaxis.set_ticklabels(np.unique(y_test_pred_decoded))
ax.set_ylabel("Actual", fontsize=14, labelpad=20)
ax.yaxis.set_ticklabels(np.unique(y_test_pred_decoded))
ax.set_title("Confusion Matrix", fontsize=14, pad=20)
plt.show()